# Notebook Module 3

<a target="_blank" href="https://colab.research.google.com/github/dannys0n/CS394-MyModulesRepo/blob/main/notebooks/03/Campus%20Agent%20Extended.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/dannys0n/CS394-MyModulesRepo/tree/main/notebooks/03/Campus%20Agent%20xtended.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

## Install pre-requisites

In [ ]:
!uv pip install openai-agents==0.4.2 gradio==5.49.1

## Set the OpenAI API Key Environment Variable

In [ ]:
import sys
import os
from dotenv import load_dotenv

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
  os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
  load_dotenv()

## Agents

In [ ]:
from agents import Agent, Runner, FileSearchTool, WebSearchTool

### Cafe Agent

In [ ]:
from agents import function_tool

@function_tool
def get_bytes_cafe_menu(date: str) -> any:
    return {
        f"{date}": {
            "daily byte": {
                "name": "Steak Quesadilla",
                "price": 12,
                "description": "Flank steak, mixed cheese in a flour tortilla served with air fried potatoes, sour cream and salsa",
            },
            "vegetarian": {
                "name": "Impossible Quesadilla",
                "price": 12,
                "description": "Impossible plant based product, mixed cheese in a flour tortilla served with air fried potatoes, sour cream and salsa",
            },
            "international": {
                "name": "Chicken Curry",
                "price": 12,
                "description": "Chicken thighs, onion, carrot, potato, curry sauce served over rice",
            },
        }
    }

In [ ]:
cafe_agent = Agent(
    name="Cafe Agent",
    instructions="You help students locate and provide information about the Bytes Cafe.",
    tools=[
        get_bytes_cafe_menu,
    ],
)

### Set Vector Store

In [ ]:
VECTOR_STORE_ID = "vs_697403d8298c81919b2938f6ef4aa347"

### Building Agent

In [ ]:
building_agent = Agent(
    name="Building Agent",
    instructions="You help students locate and provide information about buildings and rooms on campus. Be descriptive when giving locations.",
    tools=[
        FileSearchTool(
            max_num_results=3,
            vector_store_ids=[VECTOR_STORE_ID],
            include_search_results=True,
        )
    ],
)

### Course Agent

In [ ]:
course_agent = Agent(
    name="Course Agent",
    instructions="You help students find out information about courses held at DigiPen.",
    tools=[
        FileSearchTool(
            max_num_results=5,
            vector_store_ids=[VECTOR_STORE_ID],
            include_search_results=True,
        )
    ],
)

### Handbook Agent

In [ ]:
handbook_agent = Agent(
    name="Handbook Agent",
    instructions="You help students navigate the school handbook, providing information about campus policies and student conduct.",
    tools=[
        FileSearchTool(
            max_num_results=5,
            vector_store_ids=[VECTOR_STORE_ID],
            include_search_results=True,
        )
    ],
)

### Events Agent
new agent with web search tool for seaerching DigiPen related events.
I chose this as the new agent to get familiar with its web search capabilities

In [ ]:
from datetime import datetime, UTC
# current date
CURRENT_DATE = datetime.now(UTC).strftime("%Y-%m-%d")

events_agent = Agent(
    name="Events Agent",
    instructions=(
        "You are responsible for finding upcoming DigiPen Institute of Technology events.\n"
        f"Current date: {CURRENT_DATE}\n\n"
        "Rules:\n"
        "- Only report events that occur after the current date\n"
        "- Prefer official DigiPen sources (digipen.edu)\n"
        "- Use queries like: site:digipen.edu/calendar DigiPen event\n"
        "- If the calendar is dynamic, search for individual event pages\n"
        "- List events with: name, date, location (if available), and source URL\n"
        "- If events cannot be fully enumerated, say so explicitly\n"
    ),
    tools=[
        WebSearchTool(),
    ],
)

### Campus Agent

In [ ]:
agent = Agent(
    name="DigiPen Campus Agent",
    instructions="You are a helpful campus agent that can plan and execute tasks for students at DigiPen. Please be concise and accurate in handing off tasks to other agents as needed.",
    handoffs=[building_agent, course_agent, handbook_agent, cafe_agent, events_agent ],
)

## Gradio Interface

### Chat function and display of tool calls

In [ ]:
from gradio import ChatMessage

async def chat_with_agent(user_msg: str, history: list):
    messages = [{"role": msg["role"], "content": msg["content"]} for msg in history]
    messages.append({"role": "user", "content": user_msg})
    responses = []
    reply_created = False
    active_agent = None

    result = Runner.run_streamed(agent, messages)
    async for event in result.stream_events():
        if event.type == "raw_response_event":
            if event.data.type == "response.output_text.delta":
                if not reply_created:
                    responses.append(ChatMessage(role="assistant", content=""))
                    reply_created = True
                responses[-1].content += event.data.delta
        elif event.type == "agent_updated_stream_event":
            active_agent = event.new_agent.name
            responses.append(
                ChatMessage(
                    content=event.new_agent.name,
                    metadata={"title": "Agent Now Running", "id": active_agent},
                )
            )
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                if event.item.raw_item.type == "file_search_call":
                    responses.append(
                        ChatMessage(
                            content=f"Query used: {event.item.raw_item.queries}",
                            metadata={
                                "title": "File Search Completed",
                                "parent_id": active_agent,
                            },
                        )
                    )
                if event.item.raw_item.type == "web_search_call":
                    responses.append(
                        ChatMessage(
                            content=f"Query used: {event.item.raw_item}",
                            metadata={
                                "title": "Web Search Completed",
                                "parent_id": active_agent,
                            },
                        )
                    )
                else:
                    tool_name = getattr(event.item.raw_item, "name", "unknown_tool")
                    tool_args = getattr(event.item.raw_item, "arguments", {})
                    responses.append(
                        ChatMessage(
                            content=f"Calling tool {tool_name} with arguments {tool_args}",
                            metadata={"title": "Tool Call", "parent_id": active_agent},
                        )
                    )
            if event.item.type == "tool_call_output_item":
                responses.append(
                    ChatMessage(
                        content=f"Tool output: '{event.item.raw_item['output']}'",
                        metadata={"title": "Tool Output", "parent_id": active_agent},
                    )
                )
            if event.item.type == "handoff_call_item":
                responses.append(
                    ChatMessage(
                        content=f"Name: {event.item.raw_item.name}",
                        metadata={
                            "title": "Handing Off Request",
                            "parent_id": active_agent,
                        },
                    )
                )
        yield responses

### Launch Gradio

In [ ]:
import gradio as gr

demo = gr.ChatInterface(
    chat_with_agent,
    title="DigiPen Campus Agent",
    theme=gr.themes.Soft(
        primary_hue="red", secondary_hue="slate", font=[gr.themes.GoogleFont("Inter")]
    ),
    examples=[
        "What digipen specific holiday events are there? What are their dates",
        "Where can I find upcoming events at DigiPen?",
        "What events are upcoming?",
    ],
    submit_btn=True,
    flagging_mode="manual",
    flagging_options=["Like", "Spam", "Inappropriate", "Other"],
    type="messages",
    save_history=False,
)

demo.launch(share=False, debug=True)

Prompt 1:
What digipen specific holiday events are there? What are their dates

Output:
Here are the DigiPen holiday or break observances listed after today (January 28, 2026):

Winter Break: Administrative offices are closed, but the campus remains open. Break period spans December 24, 2025 through January 1, 2026, with services resuming on January 2, 2026.
Source: DigiPen calendar (digipen.edu)

Thanksgiving Observed: Recognized observance where classes are not in session, though labs remain open. Example date listed is November 27, 2025, which is before the current date and thus not upcoming.
Source: DigiPen calendar (digipen.edu)

Summary: There are no upcoming holiday-specific observances listed on the DigiPen site after January 28, 2026. The most recent holiday period (Winter Break) ended on January 1, 2026.



Prompt 2:
Where can I find upcoming events at DigiPen?

Output:
assistant
Here’s where you can find information on upcoming DigiPen Institute of Technology events:

DigiPen’s Official Events Calendar

DigiPen maintains a calendar—often titled “Calendar” or “Events”—on its official website where many on‑ and off‑campus events are listed, such as guest lectures, Company Days, workshops, and more (digipen.edu). You can access it via the student portal’s “Events” or “Calendar” section at digipen.edu.
DigiPen Near You (College Fairs & Expos)

This section lists upcoming public engagements where DigiPen participates, including college fairs and gaming expos. For example, the calendar currently includes: • Emerald City Comic Con – March 5, 2026 • PAX East – March 26, 2026 • LVL UP Expo – April 24–26, 2026 (digipen.edu).
Commencement Ceremony

The 2026 Commencement Ceremony is scheduled for Sunday, April 26, 2026, at the Lynnwood Event Center in Lynnwood, WA, with a reception to follow at DigiPen's Redmond campus (digipen.edu).
Admitted Dragon Day (for admitted prospective students)

DigiPen hosts two Admitted Dragon Day events specifically for newly admitted students: • Saturday, April 4, 2026 • Saturday, May 16, 2026 Each includes campus tours, info sessions, faculty panels, and a student social (digipen.edu).
GeekFest West 2025

An expo-style public event featuring games, cosplay, and DigiPen’s student-made games. It took place July 18–20, 2025, in Everett, WA (digipen.edu). (This one is in the past and is listed for your awareness of the kind of events DigiPen attends.)
Additional Events

While events like NextGen (a career-focused student showcase) and Global Game Jam are notable, their most recent iterations (e.g., NextGen 2025, Global Game Jam January 2024) are now past (digipen.edu).
Important note: The DigiPen calendar and "Near You" sections often list events dynamically, so there may be additional upcoming events not captured here.

If you'd like, I can help you track specific event types or set updates for new entries once they’re added.

In summary, your best bets for discovering upcoming DigiPen events are:

The official DigiPen Events or Calendar page (via student portal)
The “DigiPen Near You” page for public outreach events
Pages dedicated to major recurring events (e.g., Commencement, Admitted Dragon Day)
Let me know if you want details on any specific category or upcoming event type!

Prompt 3:
What events are upcoming?

Output:
Here are upcoming DigiPen Institute of Technology events after January 27, 2026, based on official DigiPen sources:

• Deep Dive: BFA in Digital Art and Animation Program
 Date: February 4, 2026, 3:00 p.m. PST
 Location: Online (interactive livestream)
 Description: A “Deep Dive” session offering an in-depth look at the BFA in Digital Art and Animation program, with real-time chat and Q&A.
 Source: DigiPen Online Information Sessions page (digipen.edu).

• Deep Dive: BA in Music and Sound Design Program
 Date: February 11, 2026, 3:00 p.m. PST
 Location: Online (interactive livestream)
 Description: Program-specific session covering the BA in Music and Sound Design.
 Source: DigiPen Online Information Sessions page (digipen.edu).

• Deep Dive: BA in Game Design Program
 Date: February 18, 2026, 3:00 p.m. PST
 Location: Online (interactive livestream)
 Description: Focused session providing program insights into BA in Game Design.
 Source: DigiPen Online Information Sessions page (digipen.edu).

• Deep Dive: Computer Science Programs
 Date: February 25, 2026, 3:00 p.m. PST
 Location: Online (interactive livestream)
 Description: Covers DigiPen’s computer science degree offerings in an interactive livestream.
 Source: DigiPen Online Information Sessions page (digipen.edu).

• International Student Information Session
 Date: February 27, 2026, 7:00 p.m. PST
 Location: Online (interactive webcast)
 Description: Tailored for international students, covering topics relevant to them such as resources and admissions.
 Source: DigiPen Online Information Sessions page (digipen.edu).

• Washington State Gaming Expo 2026
 Date: February 6, 2026
 Location: Puyallup, Washington (show floor)
 Description: DigiPen’s debut at the regional gaming expo, showcasing student-made games in the Expo Hall.
 Source: DigiPen Washington State Gaming Expo page (digipen.edu).

• Admitted Dragon Day
 Dates: April 4, 2026 or May 16, 2026; both Saturdays
 Location: DigiPen campus (Redmond, WA)
 Description: An on-campus event for admitted students, featuring welcome talks, faculty panels, resource fairs, tours, and an evening New Dragon Social.
 Source: DigiPen Admitted Dragon Day page (digipen.edu).

• Commencement Ceremony 2026
 Date: Sunday, April 26, 2026; 1:00–3:00 p.m. PDT (reception 3:00–6:00 p.m.)
 Location: Lynnwood Event Center, 3711 196th St. SW, Lynnwood, WA; reception at DigiPen campus, Redmond, WA
 Description: Graduation ceremony for students completing fall, spring, or summer terms. Includes guest info, arrivals, and regalia details.
 Source: DigiPen Commencement page (digipen.edu).

• Anatomy of a Student Team: Spring Edition
 Date: April 1, 2026, 3:00 p.m. PDT
 Location: Online (interactive webcast)
 Description: A session where staff and current student teams break down how they form and execute projects from scratch—games and animations.
 Source: DigiPen Online Information Sessions page (digipen.edu).

• The Social Life: Clubs and Student Life at DigiPen
 Date: April 15, 2026, 3:00 p.m. PDT
 Location: Online (interactive webcast)
 Description: An interactive webcast exploring student clubs, social avenues, and student life at DigiPen.
 Source: DigiPen Online Information Sessions page (digipen.edu).

Summary Table:

Event	Date	Location
Deep Dive: BFA in Digital Art & Animation	Feb 4, 2026	Online
Deep Dive: BA in Music & Sound Design	Feb 11, 2026	Online
Deep Dive: BA in Game Design	Feb 18, 2026	Online
Deep Dive: Computer Science Programs	Feb 25, 2026	Online
International Student Info Session	Feb 27, 2026	Online
Washington State Gaming Expo 2026	Feb 6, 2026	Puyallup, WA
Anatomy of a Student Team: Spring Edition	Apr 1, 2026	Online
The Social Life: Clubs & Student Life	Apr 15, 2026	Online
Admitted Dragon Day	Apr 4 & May 16, 2026	Redmond, WA campus
Commencement Ceremony 2026	Apr 26, 2026	Lynnwood & Redmond, WA
Note: There may be additional college fairs or regional events listed under “DigiPen Near You,” like Emerald City Comic Con or PAX East, but those lacked explicit dates tied to 2026 or were preliminary listings (digipen.edu). If you’d like more detail on any specific event, feel free to ask!

### OpenAI Traces
Created
Jan 27, 2026, 5:03 PM
ID
resp_04964d0326cb30f80069796053c3e48191a5c28a6ac3bad54b
Model
gpt-4.1-2025-04-14
Tokens
16,740 total
Tools
Web Search
Configuration
Response
text
Verbosity
medium
Instructions
System Instructions
You are responsible for finding upcoming DigiPen Institute of Technology events.
Current date: 2026-01-28

Rules:

Keep your response brief and concise. Consider token generation
Only report events that occur after the current date
Prefer official DigiPen sources (digipen.edu)
Use queries like: site:digipen.edu/calendar DigiPen event
If the calendar is dynamic, search for individual event pages
List events with: name, date, location (if available), and source URL
If events cannot be fully enumerated, say so explicitly
Input
16487t
user
What digipen specific holiday events are there? What are their dates

Function call
Arguments
transfer_to_events_agent()
Output
{"assistant": "Events Agent"}
Output
253t
Web Search
Searched the web
assistant
Here are the DigiPen holiday or break observances listed after today (January 28, 2026):

Winter Break: Administrative offices are closed, but the campus remains open. Break period spans December 24, 2025 through January 1, 2026, with services resuming on January 2, 2026.
Source: DigiPen calendar (digipen.edu)

Thanksgiving Observed: Recognized observance where classes are not in session, though labs remain open. Example date listed is November 27, 2025, which is before the current date and thus not upcoming.
Source: DigiPen calendar (digipen.edu)

Summary: There are no upcoming holiday-specific observances listed on the DigiPen site after January 28, 2026. The most recent holiday period (Winter Break) ended on January 1, 2026.



### Observations:
I found that the most inconsistent prompt was "What digipen specific holiday events are there? What are their dates"

It seems dates that are typically not freely availiable or just in the calendar section of digipen.edu site does not get accessed by the web tool. This makes the output to sometimes just guess

Sometimes an Agent never gets selected and just acts as a standard gpt-4 model. This was more consistent when using certain keywords like "event".

## Use of AI Tools (Academic Disclosure)

AI-assisted tools (ChatGPT-5.2 Auto) were used during the development of this notebook to:
- Assist in API specific syntax

All generated content was reviewed, modified, and validated by the author.
The author retains responsibility for the correctness and originality of the final submission.